21/1/2024
The following code has been edited to remove any mentions of actual production file paths

In [1]:
import pandas as pd
import os

In [2]:
FILE_PATH = "file\\path\\"
file_list = []
column_list = ["LOCATION",
"ACSVIEW_TIMESTAMP",
"AUDIT_SESSION_ID",
"MESSAGETEXT",
"ACS_SERVER",
"POLICY_SET_NAME",
"PASSED",
"NAS_IP_ADDRESS",
"IDENTITY_GROUP",
"SELECTED_AZN_PROFILES",
"NETWORK_DEVICE_NAME",
"AUTHENTICATION_METHOD",
"CTS_SECURITY_GROUP",
"NAS_PORT_ID",
"DEVICE_TYPE",
"USER_NAME",
"AUTHORIZATION_RULE",
"RESPONSE_TIME",
"CALLING_STATION_ID",
"IDENTITY_STORE",
"SERVICE_TYPE"]

combined_df = pd.DataFrame()

In [3]:
for root, dirs, files in os.walk(FILE_PATH):
   for name in files:
       file_list.append(os.path.join(root, name))

In [5]:
for file in file_list:
    df1 = pd.read_csv(file, usecols=column_list)
    
    combined_df = pd.concat([combined_df, df1], ignore_index=True)

In [ ]:
combined_df = combined_df.sort_values("ACSVIEW_TIMESTAMP")
combined_df["AUTHORIZATION_RULE"].fillna("''", inplace = True)
combined_df

In [8]:
def endpoint_count(df):
    
#     drop_col = ["'MDM_SERVER_NAME'","'FAILURE_REASON'","'ID'",
#                 "'NAS_IPV6_ADDRESS'","'MSE_SERVER_NAME'","'AUTHENTICATION_PROTOCOL'",
#                 "'CREDENTIAL_CHECK'","'MSE_RESPONSE_TIME'","'ENDPOINTMATCHEDPROFILE'","'MAPLOCATION'"]
    
    auth_filter = ["'Authentication succeeded'", "'Authentication failed'"]

#     endpoint_df = df.drop(columns=drop_col)
#     endpoint_df = df[df["MESSAGETEXT"].isin(auth_filter)]
    endpoint_df = df.drop_duplicates(subset = "USER_NAME")
    
    endpoint_df.to_excel(FILE_PATH+ 'Total Endpoints ' +str(len(endpoint_df.index))+'.xlsx', index=False)

In [9]:
def employee_byod(df):
    # 'CIBC SQ Employee BYOD_IN' , 
    byod_df = df[df["AUTHORIZATION_RULE"].str.contains("BYOD", case=False)]
    byod_df = byod_df[~byod_df["AUTHORIZATION_RULE"].str.contains("redirect", case=False)]
    byod_df = byod_df[byod_df["MESSAGETEXT"].str.contains("Authorize-Only succeeded", case=False)]
    
    byod_df.to_excel(FILE_PATH+ 'BYOD Users - Graph.xlsx', index=False, sheet_name="Raw Data")

In [10]:
def cmc_guest(df):
    guest_df = df[df["AUTHORIZATION_RULE"].str.contains("CMC-GUEST", case=False)]
    guest_df = guest_df[~guest_df["AUTHORIZATION_RULE"].str.contains("redirect", case=False)]
    guest_df = guest_df[guest_df["SELECTED_AZN_PROFILES"].str.contains("PermitAccess", case=False)]
    guest_df = guest_df[guest_df["MESSAGETEXT"].str.contains("Authorize-Only succeeded", case=False)]
    
    guest_df.to_excel(FILE_PATH+ 'Guest Users - Graph.xlsx', index=False, sheet_name="Raw Data")

In [11]:
def sq_hotspot(df):
    hotspot_df = df[df["AUTHORIZATION_RULE"].str.contains("Hotspot", case=False)]
    hotspot_df = hotspot_df[~hotspot_df["AUTHORIZATION_RULE"].str.contains("redirect", case=False)]
#     hotspot_df = hotspot_df[hotspot_df["'SELECTED_AZN_PROFILES'"].str.contains("PermitAccess", case=False)]
    hotspot_df = hotspot_df[hotspot_df["MESSAGETEXT"].str.contains("Authorize-Only succeeded", case=False)]
#     print(len(hotspot_df.index))
    
    hotspot_df.to_excel(FILE_PATH+ 'Pre Auth Hotspot.xlsx', index=False, sheet_name="Raw Data")

In [12]:
def cibc_events(df):
    events_df = df[df["AUTHORIZATION_RULE"].str.contains("EVENT", case=False)]
    events_df = events_df[~events_df["AUTHORIZATION_RULE"].str.contains("redirect", case=False)]
    events_df = events_df[events_df["SELECTED_AZN_PROFILES"].str.contains("PermitAccess", case=False)]
    events_df = events_df[events_df["MESSAGETEXT"].str.contains("Authorize-Only succeeded", case=False)]
    
    events_df.to_excel(FILE_PATH+ 'CIBC Events - Graph.xlsx', index=False, sheet_name="Raw Data")

In [13]:
endpoint_count(combined_df)
employee_byod(combined_df)
cmc_guest(combined_df)
sq_hotspot(combined_df)
cibc_events(combined_df)